## COVID-19 Statistics in India - Visualization

In [2]:
# Importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [12]:
# Defining custom parameters
interpolate_limit = 10
timebox = '2020.05'

In [13]:
# Importing the dataset and generating necessary references
raw_data = pd.read_excel(f'/content/drive/My Drive/Covid/Covid{timebox}.xlsx', sheet_name = 'Stats')
state_cd = pd.read_excel(f'/content/drive/My Drive/Covid/Covid{timebox}.xlsx', sheet_name = 'Legend')
states = state_cd['Code'].tolist()
dates = list(set(raw_data['Date'].tolist()))
dates.sort()
raw_data = raw_data.sort_values(['State', 'Date'], ascending = [True, True])
dayscount = len(raw_data[raw_data['State'] == 'AN'])

In [ ]:
# Preprocessing the data
covid = pd.DataFrame(columns= ['Date', 'Checkpoint', 'State', 'Cases'])
for state in states:
  temp_df = raw_data[raw_data['State'] == state].reset_index()
  for i in range(len(temp_df)):
    for j in range(interpolate_limit):
      if j == 0:
        tempdic = {'Date': temp_df['Date'][i],
                   'Checkpoint': 0,
                   'State': temp_df['State'][i],
                   'Cases': float(temp_df['Cases'][i])
                   }
        covid = covid.append(tempdic, ignore_index = True)
      elif j != 0 and i < (len(temp_df) - 1):
        tempdic = {'Date': temp_df['Date'][i],
                   'Checkpoint': j,
                   'State': temp_df['State'][i],
                   'Cases': np.nan
                   }
        covid = covid.append(tempdic, ignore_index = True)    
covid = covid.interpolate()

In [ ]:
covid = covid.sort_values(['Date', 'Checkpoint', 'State'], ascending = [True, True, True])
x = 0
for i in range(dayscount):
  dayt = f"{dates[i].strftime('%b')} {dates[i].strftime('%d')}\n{dates[i].strftime('%Y')}"
  for j in range(interpolate_limit):
    x += 1
    plot_df = covid[(covid['Date'] == dates[i]) & (covid['Checkpoint'] == j)]
    fig, ax = plt.subplots(figsize = (40, 20))
    fig.text(.90, .75, dayt, fontsize = 40, color = '#000000', ha = 'right', va = 'top', alpha = 1)
    sns.set_context("paper", rc = {"font.size": 3, "axes.titlesize": 50, "axes.labelsize": 35})
    sns.barplot(x = plot_df['State'], y = plot_df['Cases'], data = plot_df, label = 'GGGG')
    sns.despine(top = True, right = True)
    plt.title("Corona Virus in India : Daily Active Cases Trend")
    plt.xticks(size = 25)
    plt.yticks(size = 25)
    plt.savefig(f'/content/drive/My Drive/Covid/{timebox}/Img{x}.jpg')
    plt.close()
    if i == (dayscount - 1):
      break
  if (i + 1) % 5 == 0:
    print(f'Plot generation done upto {dates[i]}')